# sub-n-classes, Tsallis Entropy labelling, looped

In [9]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression as LR
from sklearn.ensemble import RandomForestClassifier as RF
import time
from numpy.random import *

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action = 'ignore', category = FutureWarning)

In [10]:
def randints_nodup(k):
    ns = []
    while (len(ns) < k):
        n = randint(0, 10)
        if not n in ns:
            ns.append(n)
    return list(ns)

In [11]:
# entropy labelling
def tsallis_ent_label(q, probas, s_cls):
    elements = np.power(probas, q - 1)
    # thrshld in tsallis entropy model
    ts_thrshld = np.sum(np.power(probas, q))
    labels = np.array([s_cls[i] for i, e in enumerate(elements) if e >= ts_thrshld])
    
    return labels

#### labelling and evaluating them

In [12]:
# labelling and evaluating them
def tsallis_ent_scls_eval(q, classes_num, orig_A, lim_A):

    s_cls = randints_nodup(classes_num)
    s_cls.sort()
    #print(s_cls)

    # extract dataset of chosen classes
    trn_imgs = [img for i, img in enumerate(train_imgs) if train_labels[i] in s_cls]
    trn_labels = [label for label in train_labels if label in s_cls]

    tst_imgs = [img for i, img in enumerate(test_imgs) if test_labels[i] in s_cls]
    tst_labels = [label for label in test_labels if label in s_cls]

    # generate an annotator
    a1_model = LR().fit(trn_imgs[:orig_A], trn_labels[:orig_A])
    a1_proba = a1_model.predict_proba(trn_imgs[orig_A:orig_A + lim_A])

    # entropy labelling
    mul_labels = [tsallis_ent_label(q, probas, s_cls) for probas in a1_proba]
    
    # labels score evaluation
    score = 0
    for labels, t_label in zip(mul_labels, trn_labels[orig_A:orig_A + lim_A]):
        for l in labels:
            if (l == t_label):
                score += 1

    m_labels = []
    for labels in mul_labels:
         [m_labels.append(l) for l in labels]                
                    
    return (len(m_labels)/lim_A, score*100/len(m_labels), score*100/lim_A) 

## MNIST

In [5]:
from keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_imgs = np.array([x.ravel() for x in train_images])
test_imgs = np.array([y.ravel() for y in test_images])

img_SIZE = train_images.shape[1]*train_images.shape[2]

Using TensorFlow backend.


### labels evalutaiton

In [20]:
orig_A1, lim_A1 = 2000, 2000
q = 3/2
mnist_evals = []
for j in range(2, 11):
    a, b, c = 0, 0, 0
    if j == 10:
        sample_lnum, sample_lqual, sample_lqual2 = tsallis_ent_scls_eval(q, j, orig_A1, lim_A1)
        mnist_evals.append((sample_lnum, sample_lqual, sample_lqual2))
    else:    
        for _ in range(10):
            sample_lnum, sample_lqual, sample_lqual2 = tsallis_ent_scls_eval(q, j, orig_A1, lim_A1)
            a += sample_lnum
            b += sample_lqual
            c += sample_lqual2
        mnist_evals.append((a/10, b/10, c/10))
print(f"evaluation of labels genereated for mnist; (# of labels per image, labels quality [%], lebels quality as groups [%])\n{mnist_evals}")

evaluation of labels genereated for mnist; (# of labels per image, labels quality [%], lebels quality as groups [%])
[(1.0, 98.29499999999999, 98.29499999999999), (1.0002499999999999, 95.59151395081679, 95.615), (1.0058500000000001, 90.13900838291002, 90.66), (1.0080500000000001, 91.22951357420432, 91.96), (1.0152, 88.83585469795497, 90.18000000000002), (1.0198, 87.85993767683763, 89.595), (1.02665, 85.37827338993625, 87.65000000000002), (1.03605, 83.64809418463929, 86.66), (1.0455, 82.30511716881875, 86.05)]


## CIFAR 10

In [13]:
from keras.datasets import cifar10

(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# number of data samples
train_N, test_N = train_images.shape[0], test_images.shape[0]
# data normalisation and vecotrise
train_imgs, test_imgs = train_images.reshape(train_N, -1)/255, test_images.reshape(test_N, -1)/255
# labels ravel
train_labels, test_labels = train_labels.ravel(), test_labels.ravel()

img_SIZE = train_images.shape[1]*train_images.shape[2]*train_images.shape[3]

### labels evalutaiton

In [8]:
orig_A2, lim_A2 = 2000, 2000
q = 3/2
cifar10_evals = []
for j in range(2, 11):
    a, b, c = 0, 0, 0
    if j == 10:
        sample_lnum, sample_lqual, sample_lqual2 = tsallis_ent_scls_eval(q, j, orig_A2, lim_A2)
        cifar10_evals.append((sample_lnum, sample_lqual, sample_lqual2))
    else:    
        for _ in range(10):
            sample_lnum, sample_lqual, sample_lqual2 = tsallis_ent_scls_eval(q, j, orig_A2, lim_A2)
            a += sample_lnum
            b += sample_lqual
            c += sample_lqual2
        cifar10_evals.append((a/10, b/10, c/10))
print(f"evaluation of labels genereated for cifar10; (# of labels per image, labels quality [%], lebels quality as groups [%])\n{cifar10_evals}")

evaluation of labels genereated for cifar10; (# of labels per image, labels quality [%], lebels quality as groups [%])
[(1.0, 78.24499999999999, 78.24499999999999), (1.0372499999999998, 60.0922117553636, 62.19500000000001), (1.0888000000000002, 52.546105159347725, 57.11), (1.1545, 44.307437685247876, 51.12), (1.20525, 39.55322829383922, 47.65), (1.2684, 34.36298459645821, 43.584999999999994), (1.3268999999999997, 30.80721330537512, 40.870000000000005), (1.37655, 27.715548538067765, 38.150000000000006), (1.4115, 26.071555083244775, 36.8)]


## CIFAR100

In [14]:
from keras.datasets import cifar100

(train_images, train_labels), (test_images, test_labels) = cifar100.load_data(label_mode = "fine")

# number of data samples
train_N, test_N = train_images.shape[0], test_images.shape[0]
# data normalisation and vecotrise
train_imgs, test_imgs = train_images.reshape(train_N, -1)/255, test_images.reshape(test_N, -1)/255
# labels ravel
train_labels, test_labels = train_labels.ravel(), test_labels.ravel()

img_SIZE = train_images.shape[1]*train_images.shape[2]*train_images.shape[3]

### labels evalutaiton

In [15]:
orig_A3, lim_A3 = 200, 200
q = 3/2
cifar100_evals = []
for j in range(2, 11):
    a, b, c = 0, 0, 0
    if j == 10:
        sample_lnum, sample_lqual, sample_lqual2 = tsallis_ent_scls_eval(q, j, orig_A3, lim_A3)
        cifar100_evals.append((sample_lnum, sample_lqual, sample_lqual2))
    else:    
        for _ in range(10):
            sample_lnum, sample_lqual, sample_lqual2 = tsallis_ent_scls_eval(q, j, orig_A3, lim_A3)
            a += sample_lnum
            b += sample_lqual
            c += sample_lqual2
        cifar100_evals.append((a/10, b/10, c/10))
print(f"evaluation of labels genereated for cifar100; (# of labels per image, labels quality [%], lebels quality as groups [%])\n{cifar100_evals}")

evaluation of labels genereated for cifar100; (# of labels per image, labels quality [%], lebels quality as groups [%])
[(1.0, 75.0, 75.0), (1.017, 60.25056739060599, 61.25), (1.0805, 53.52036013254544, 57.75), (1.1355, 45.261531662373024, 51.35), (1.2025, 38.57491714435389, 46.35), (1.2685, 34.58064583939547, 43.8), (1.2865, 33.03962271553248, 42.35), (1.3505, 26.780882914526387, 36.15), (1.365, 27.106227106227106, 37.0)]


#### Tsallis entropy based labellingの結果ベタ打ち

In [16]:
mnist_evals = [(1.0, 98.29499999999999, 98.29499999999999), (1.0002499999999999, 95.59151395081679, 95.615), (1.0058500000000001, 90.13900838291002, 90.66), (1.0080500000000001, 91.22951357420432, 91.96), (1.0152, 88.83585469795497, 90.18000000000002), (1.0198, 87.85993767683763, 89.595), (1.02665, 85.37827338993625, 87.65000000000002), (1.03605, 83.64809418463929, 86.66), (1.0455, 82.30511716881875, 86.05)]
cifar10_evals = [(1.0, 78.24499999999999, 78.24499999999999), (1.0372499999999998, 60.0922117553636, 62.19500000000001), (1.0888000000000002, 52.546105159347725, 57.11), (1.1545, 44.307437685247876, 51.12), (1.20525, 39.55322829383922, 47.65), (1.2684, 34.36298459645821, 43.584999999999994), (1.3268999999999997, 30.80721330537512, 40.870000000000005), (1.37655, 27.715548538067765, 38.150000000000006), (1.4115, 26.071555083244775, 36.8)]
cifar100_evals = [(1.0, 75.0, 75.0), (1.017, 60.25056739060599, 61.25), (1.0805, 53.52036013254544, 57.75), (1.1355, 45.261531662373024, 51.35), (1.2025, 38.57491714435389, 46.35), (1.2685, 34.58064583939547, 43.8), (1.2865, 33.03962271553248, 42.35), (1.3505, 26.780882914526387, 36.15), (1.365, 27.106227106227106, 37.0)]

#### Shannon entropy based labellingの結果ベタ打ち

In [17]:
shannon_mnist_evals = [(0.9356500000000001, 97.59112842711961, 91.44), (1.00055, 95.09837104519026, 95.15), (1.0051999999999999, 92.11736652570067, 92.585), (1.0105, 91.63631865089286, 92.58500000000001), (1.0141499999999997, 90.18507482103416, 91.45), (1.0228, 87.52875104060001, 89.52000000000001), (1.0335999999999999, 85.03314578957648, 87.88499999999999), (1.04305, 83.77746449706856, 87.38), (1.0515, 82.1207798383262, 86.35)]
shannon_cifar10_evals = [(1.0, 74.75, 74.75), (1.0503, 59.12454317334923, 61.989999999999995), (1.1259000000000001, 52.07008511489069, 58.395), (1.21395, 44.659508244866444, 54.17), (1.2910499999999998, 38.921387720990715, 50.225), (1.37705, 33.05457607486076, 45.485), (1.45695, 30.184617088821017, 43.97), (1.53715, 26.949681416477638, 41.42), (1.573, 25.33375715193897, 39.85)]
shannon_cifar100_evals = [(1.0, 74.05, 74.05), (1.0544999999999998, 62.104241102486604, 65.35), (1.1214999999999997, 51.47451323040567, 57.7), (1.1934999999999998, 44.192441242368616, 52.65), (1.2449999999999997, 42.06936819425348, 52.25), (1.355, 32.95733722136263, 44.65), (1.4009999999999998, 30.70275968857884, 43.0), (1.4565000000000001, 26.245987142981033, 38.2), (1.505, 25.58139534883721, 38.5)]

#### 1/K based labellingの結果ベタ打ち

In [18]:
oneKth_mnist_evals = [(1.0, 98.535, 98.535), (1.0337999999999998, 93.474425718079, 96.59), (1.0581, 90.81419007456978, 96.01), (1.0956000000000001, 86.1299731028797, 94.28), (1.1239000000000001, 82.26485567364804, 92.39500000000001), (1.1456, 80.36772302436852, 91.985), (1.1787, 76.9199874518848, 90.645), (1.2041499999999998, 74.92200535609274, 90.2), (1.2255, 73.19461444308446, 89.7)]
oneKth_cifar10_evals = [(1.0, 74.89500000000001, 74.89500000000001), (1.2692, 55.37420648812955, 70.145), (1.49495, 45.74459316392563, 68.3), (1.6934, 39.62396687129409, 67.05499999999999), (1.9530999999999998, 32.21940547853991, 62.86), (2.1416, 28.74886685872645, 61.55), (2.3503999999999996, 25.600694800298573, 60.154999999999994), (2.5581, 22.96395128959202, 58.739999999999995), (2.712, 21.109882005899706, 57.25)]
oneKth_cifar100_evals = [(1.0, 77.2, 77.2), (1.24, 58.172364364922245, 72.0), (1.4460000000000002, 44.860119469499764, 64.75), (1.6415, 39.18608024073104, 64.2), (1.8370000000000002, 31.914771437344246, 58.5), (2.066, 28.362822089935, 58.5), (2.2589999999999995, 24.41651839267302, 55.15), (2.4054999999999995, 21.666044289564223, 52.1), (2.525, 20.594059405940595, 52.0)]

In [21]:
# Tsallis Entropy
lnum_mnist = [e[0] for e in mnist_evals]
lqual_mnist = [e[1] for e in mnist_evals]

lnum_cifar10 = [e[0] for e in cifar10_evals]
lqual_cifar10 = [e[1] for e in cifar10_evals]

lnum_cifar100 = [e[0] for e in cifar100_evals]
lqual_cifar100 = [e[1] for e in cifar100_evals]

# Shannon Entropy
lnum_mnist_shannon = [e[0] for e in shannon_mnist_evals]
lqual_mnist_shannon = [e[1] for e in shannon_mnist_evals]

lnum_cifar10_shannon = [e[0] for e in shannon_cifar10_evals]
lqual_cifar10_shannon = [e[1] for e in shannon_cifar10_evals]

lnum_cifar100_shannon = [e[0] for e in shannon_cifar100_evals]
lqual_cifar100_shannon = [e[1] for e in shannon_cifar100_evals]

#### labels acc

In [30]:
plt.figure(dpi = 100)
plt.title("Quality of labels; Tsallis Entropy vs Shannon Entropy")

# tsallis Entropy
plt.plot([i for i in range(2, 11)], lqual_mnist, marker = "o", color = "r", label = "Tsallis MNIST")
plt.plot([i for i in range(2, 11)], lqual_cifar10, marker = "o", color = "r", label = "Tsallis CIFAR10")
# plt.plot([i for i in range(2, 11)], lqual_cifar100, marker = "o", color = "g", label = "Tsallis CIFAR100")

# shannon
plt.plot([i for i in range(2, 11)], lqual_mnist_shannon, marker = "x", color = "b", label = "Shannon MNIST")
plt.plot([i for i in range(2, 11)], lqual_cifar10_shannon, marker = "x", color = "b", label = "Shannon CIFAR10")
# plt.plot([i for i in range(2, 11)], lqual_cifar100_shannon, marker = "x", color = "k", label = "Shannon CIFAR100")

plt.xlabel("Num of sub-classes")
plt.ylabel("Average label quality of labels generated")	
plt.legend(loc = "best")
plt.grid(True)
plt.savefig("tsallis-vs-shannon_labels-acc.pdf")

#### ave num of labels

In [28]:
plt.figure(dpi = 100)
plt.title("Average number of labels; Tsallis Entropy vs Shannon Entropy")
# Shannon Entropy
plt.plot([i for i in range(2, 11)], lnum_mnist, marker = "o", color = "r", label = "Tsallis MNIST")
plt.plot([i for i in range(2, 11)], lnum_cifar10, marker = "o", color = "r", label = "Tsallis CIFAR10")
#plt.plot([i for i in range(2, 11)], lnum_cifar100, marker = "o", color = "g", label = "Tsallis CIFAR100")
# 1/K
plt.plot([i for i in range(2, 11)], lnum_mnist_shannon, marker = "x", color = "b", label = "Shannon MNIST")
plt.plot([i for i in range(2, 11)], lnum_cifar10_shannon, marker = "x", color = "b", label = "Shannon CIFAR10")
#plt.plot([i for i in range(2, 11)], lnum_cifar100_shannon, marker = "x", color = "k", label = "1/K CIFAR100")

plt.xlabel("Num of sub-classes")
plt.ylabel("Average number of labels per image")	
plt.legend(loc = "best")
plt.grid(True)
plt.savefig("tsallis-vs-shannon_labels-num.pdf")